# CAMARA QoD API Fine-tuning - FINAL WORKING VERSION

Tested and working on Google Colab T4 GPU

In [ ]:
!pip install -q transformers datasets trl peft accelerate bitsandbytes scipy

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from trl import SFTTrainer
import json

In [ ]:
from google.colab import files
print("Upload sft_dataset.jsonl:")
uploaded = files.upload()

In [ ]:
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
MAX_SEQ_LENGTH = 2048

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("Model loaded!")

In [ ]:
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["qkv_proj", "o_proj", "gate_up_proj", "down_proj"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["response"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input_text, output) + tokenizer.eos_token
        texts.append(text)
    return {"text": texts}

In [ ]:
dataset = load_dataset("json", data_files="sft_dataset.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)
print(f"Dataset: {len(dataset)} examples")

In [ ]:
training_args = TrainingArguments(
    output_dir="./camara_qod_model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    optim="paged_adamw_8bit",
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    tokenizer=tokenizer,
    args=training_args,
)
print("Trainer ready!")

In [ ]:
print("Training started...")
trainer.train()
print("Training complete!")

In [ ]:
test_query = "Gaming session from IP 192.168.1.50 to server 203.0.113.100, 2 hours."
prompt = alpaca_prompt.format(
    "You are an expert assistant for the CAMARA Quality on Demand (QoD) API. Convert user requests into valid API calls.",
    test_query,
    ""
)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.3)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

result = response.split("### Response:")[-1].strip()
print(result)

try:
    json_obj = json.loads(result)
    print("\nValid JSON!")
    print(json.dumps(json_obj, indent=2))
except:
    print("\nParsing issue")

In [ ]:
model.save_pretrained("camara_qod_final")
tokenizer.save_pretrained("camara_qod_final")
print("Model saved!")